# Obspy deals with station inventory XML files

## Fei Zhang

2020-02-11

## How to build a stationxml file (with extra elements) from scratch

In [1]:
# https://docs.obspy.org/tutorial/code_snippets/stationxml_file_from_scratch.html

import obspy
from obspy import Inventory, UTCDateTime
from obspy.core.inventory import Inventory, Network, Station, Channel, Site
from obspy.clients.nrl import NRL

from obspy.core.util import AttribDict


# We'll first create all the various objects. These strongly follow the
# hierarchy of StationXML files.
inv = Inventory(
    # We'll add networks later.
    networks=[],
    # The source should be the id whoever create the file.
    source="ObsPy-Tutorial")

net = Network(
    # This is the network code according to the SEED standard.
    code="XX",
    # A list of stations. We'll add one later.
    stations=[],
    description="A test stations.",
    # Start-and end dates are optional.
    start_date=obspy.UTCDateTime(2016, 1, 2))

sta = Station(
    # This is the station code according to the SEED standard.
    code="ABC",
    latitude=1.0,
    longitude=2.0,
    elevation=345.0,
    creation_date=obspy.UTCDateTime(2016, 1, 2),
    site=Site(name="First station"),
    )

cha = Channel(
    # This is the channel code according to the SEED standard.
    code="HHZ",
    # This is the location code according to the SEED standard.
    location_code="",
    # Note that these coordinates can differ from the station coordinates.
    latitude=1.0,
    longitude=2.0,
    elevation=345.0,
    depth=10.0,
    azimuth=0.0,
    dip=-90.0,
    sample_rate=200)

# By default this accesses the NRL online. Offline copies of the NRL can
# also be used instead
nrl = NRL()
# The contents of the NRL can be explored interactively in a Python prompt,
# see API documentation of NRL submodule:
# http://docs.obspy.org/packages/obspy.clients.nrl.html
# Here we assume that the end point of data logger and sensor are already
# known:
response = nrl.get_response( # doctest: +SKIP
    sensor_keys=['Streckeisen', 'STS-1', '360 seconds'],
    datalogger_keys=['REF TEK', 'RT 130 & 130-SMA', '1', '200'])


# Now tie it all together.
cha.response = response
sta.channels.append(cha)
net.stations.append(sta)
inv.networks.append(net)

# And finally write it to a StationXML file. We also force a validation against
# the StationXML schema to ensure it produces a valid StationXML file.
#
# Note that it is also possible to serialize to any of the other inventory
# output formats ObsPy supports.
inv.write("station.xml", format="stationxml", validate=True)


# Fei's element
#extra="GPScorrection2.5"

extra = AttribDict({
           'GPSClockCorrection': {
                'value': True,
                'namespace': 'http://some-page.de/xmlns/1.0',
                'attrib': {
                  '{http://some-page.de/xmlns/1.0}Description': 'Fei added new Block QA/QC time correction data',
                  '{http://some-page.de/xmlns/1.0}date': "2012-11-27",
                  '{http://some-page.de/xmlns/1.0}value_sec':"1.0398489"
                }
            },
          'GPSClockCorrection': {
                'value': 'Fei added new Block QA/QC time correction data',
                'namespace': 'http://some-page.de/xmlns/1.0',
                'attrib': {
                  '{http://some-page.de/xmlns/1.0}date': "2012-11-28",
                  '{http://some-page.de/xmlns/1.0}value_sec':"0.9498489"
                }
            },
    
           'my_tag_2': {
                'value': u'True',
                'namespace': 'http://some-page.de/xmlns/1.0'
            },
           'my_tag_3': {
                'value': 1,
                'namespace': 'http://some-page.de/xmlns/1.0'
            },
           'my_tag_4': {
                'value': UTCDateTime('2013-01-02T13:12:14.600000Z'),
                'namespace': 'http://test.org/xmlns/0.1'
            },
           'my_attribute': {
                'value': 'my_attribute_value',
                'type': 'attribute',
                'namespace': 'http://test.org/xmlns/0.1'
            }
        })

inv.networks[0].stations[0].extra = extra
inv.write('my_inventory_extra.xml', format='STATIONXML',
          nsmap={'my_ns': 'http://test.org/xmlns/0.1',
                 'somepage_ns': 'http://some-page.de/xmlns/1.0'})

In [2]:
from obspy import read_inventory

inv = read_inventory('my_inventory_extra.xml')
print(inv.networks[0].stations[0].extra)

AttribDict({'GPSClockCorrection': AttribDict({'namespace': 'http://some-page.de/xmlns/1.0', 'value': 'Fei added new Block QA/QC time correction data', 'attrib': {'{http://some-page.de/xmlns/1.0}value_sec': '0.9498489', '{http://some-page.de/xmlns/1.0}date': '2012-11-28'}}), 'my_tag_2': AttribDict({'value': 'True', 'namespace': 'http://some-page.de/xmlns/1.0'}), 'my_tag_3': AttribDict({'value': '1', 'namespace': 'http://some-page.de/xmlns/1.0'}), 'my_tag_4': AttribDict({'value': '2013-01-02T13:12:14.600000Z', 'namespace': 'http://test.org/xmlns/0.1'}), 'my_attribute': AttribDict({'type': 'attribute', 'namespace': 'http://test.org/xmlns/0.1', 'value': 'my_attribute_value'})})


# custom defined tags in StationXML with the Obspy Inventory¶

https://docs.obspy.org/tutorial/code_snippets/stationxml_custom_tags.html
    

In [3]:
from obspy import Inventory, UTCDateTime
from obspy.core.inventory import Network
from obspy.core.util import AttribDict

extra = AttribDict({
           'my_tag': {
                'value': True,
                'namespace': 'http://some-page.de/xmlns/1.0',
                'attrib': {
                  '{http://some-page.de/xmlns/1.0}my_attrib1': '123.4',
                  '{http://some-page.de/xmlns/1.0}my_attrib2': '567'
                }
            },
           'my_tag_2': {
                'value': u'True',
                'namespace': 'http://some-page.de/xmlns/1.0'
            },
           'my_tag_3': {
                'value': 1,
                'namespace': 'http://some-page.de/xmlns/1.0'
            },
           'my_tag_4': {
                'value': UTCDateTime('2013-01-02T13:12:14.600000Z'),
                'namespace': 'http://test.org/xmlns/0.1'
            },
           'my_attribute': {
                'value': 'my_attribute_value',
                'type': 'attribute',
                'namespace': 'http://test.org/xmlns/0.1'
            }
        })

inv = Inventory([Network('XX')], 'XX')
inv[0].extra = extra
inv.write('my_inventory.xml', format='STATIONXML',
          nsmap={'my_ns': 'http://test.org/xmlns/0.1',
                 'somepage_ns': 'http://some-page.de/xmlns/1.0'})

In [4]:
from obspy import Inventory
from obspy.core.inventory import Network
from obspy.core.util import AttribDict

ns = 'http://some-page.de/xmlns/1.0'
# fxz547@vdi-n25 /g/data/ha3/Passive/SHARED_DATA/GPS_Clock/corrections
# $ head 7D.DE43_clock_correction.csv
csv_data = """ 
net,sta,date,clock_correction
7D,DE43,2012-11-27,1.0398489013215846
7D,DE43,2012-11-28,0.9408504322549281
7D,DE43,2012-11-29,0.8418519631882714
7D,DE43,2012-11-30,0.7428534941216148
7D,DE43,2012-12-01,0.6438550250549583
7D,DE43,2012-12-02,0.5448565559883017
7D,DE43,2012-12-03,0.445858086921645
7D,DE43,2012-12-04,0.3468596178549885
7D,DE43,2012-12-05,0.247861148788332
"""

my_tag = AttribDict()
my_tag.namespace = ns
my_tag.value = AttribDict()

my_tag.value.my_gpsclockcorrection = AttribDict()
my_tag.value.my_gpsclockcorrection.namespace = ns
my_tag.value.my_gpsclockcorrection.value = csv_data

my_tag.value.my_nested_tag2 = AttribDict()
my_tag.value.my_nested_tag2.namespace = ns
my_tag.value.my_nested_tag2.value = True

inv = Inventory([Network('XX')], 'XX')
inv[0].extra = AttribDict()
inv[0].extra.my_tag = my_tag
inv.write('my_inventory_nested.xml', format='STATIONXML',
          nsmap={'somepage_ns': 'http://some-page.de/xmlns/1.0'})

In [5]:
from obspy import read_inventory

inv = read_inventory('my_inventory_nested.xml')
print(inv[0].extra.my_tag.value.my_gpsclockcorrection.value)
print(inv[0].extra.my_tag.value.my_nested_tag2.value)

 
net,sta,date,clock_correction
7D,DE43,2012-11-27,1.0398489013215846
7D,DE43,2012-11-28,0.9408504322549281
7D,DE43,2012-11-29,0.8418519631882714
7D,DE43,2012-11-30,0.7428534941216148
7D,DE43,2012-12-01,0.6438550250549583
7D,DE43,2012-12-02,0.5448565559883017
7D,DE43,2012-12-03,0.445858086921645
7D,DE43,2012-12-04,0.3468596178549885
7D,DE43,2012-12-05,0.247861148788332

True


In [6]:
print(inv)

Inventory created at 2020-02-20T22:32:13.630242Z
	Created by: ObsPy 1.1.1
		    https://www.obspy.org
	Sending institution: XX
	Contains:
		Networks (1):
			XX
		Stations (0):

		Channels (0):



# Modify our station xml

Given original input station xml file, we want to read in and add extra metadada to it, 
then write out a new station xml file with the extra metadata stored.

In [8]:
# print (my_inv.networks[0].stations)

In [9]:
print(selected_inv)

Inventory created at 2017-09-05T16:57:36.000000Z
	Created by: ObsPy 1.0.2
		    https://www.obspy.org
	Sending institution: Geoscience Australia
	Contains:
		Networks (1):
			7D
		Stations (1):
			7D.DE43 (DE43)
		Channels (3):
			7D.DE43..BHZ, 7D.DE43..BHN, 7D.DE43..BHE


# Reading the new modified stationxml file and make use of the gps correcton CSV data

In [10]:
our_new_station_xml= 'modified_inventory_select.xml'


our_inv = read_inventory(our_new_station_xml)
# print(our_inv.networks[0].stations[0].extra)

csv_str = our_inv.networks[0].stations[0].extra.gpsclockcorrection.value

In [11]:
print (csv_str)

net,sta,date,clock_correction
7D,DE43,2012-11-27,1.0398489013215846
7D,DE43,2012-11-28,0.9408504322549281
7D,DE43,2012-11-29,0.8418519631882714
7D,DE43,2012-11-30,0.7428534941216148
7D,DE43,2012-12-01,0.6438550250549583
7D,DE43,2012-12-02,0.5448565559883017
7D,DE43,2012-12-03,0.445858086921645
7D,DE43,2012-12-04,0.3468596178549885
7D,DE43,2012-12-05,0.247861148788332
7D,DE43,2012-12-06,0.14886267972167544
7D,DE43,2012-12-07,0.04986421065501878
7D,DE43,2012-12-08,-0.049134258411637766
7D,DE43,2012-12-09,-0.14813272747829442
7D,DE43,2012-12-10,-0.24713119654495086
7D,DE43,2012-12-11,-0.3461296656116075
7D,DE43,2012-12-12,-0.4451281346782642
7D,DE43,2012-12-13,-0.5441266037449206
7D,DE43,2012-12-14,-0.6431250728115772
7D,DE43,2012-12-15,-0.7421235418782337
7D,DE43,2012-12-16,-0.8411220109448904
7D,DE43,2012-12-17,-0.9401204800115468
7D,DE43,2012-12-18,-1.0391189490782036
7D,DE43,2012-12-19,-1.13811741814486
7D,DE43,2012-12-20,-1.2371158872115164
7D,DE43,2012-12-21,-1.3361143562781734
7D,D

In [12]:

import pandas as pd

In [13]:

df = pd.DataFrame([x.split(',') for x in csv_str.split('\n')][2:])
print(df.head())

    0     1           2                   3
0  7D  DE43  2012-11-28  0.9408504322549281
1  7D  DE43  2012-11-29  0.8418519631882714
2  7D  DE43  2012-11-30  0.7428534941216148
3  7D  DE43  2012-12-01  0.6438550250549583
4  7D  DE43  2012-12-02  0.5448565559883017


In [14]:
# This should be replaced by the next code using tempfile 

# CSV_FILE_NAME = 'temp_file.csv'  # Consider creating temp file, look example below
# with open(CSV_FILE_NAME, 'w') as outfile:
#     outfile.write(csv_str)
# df = pd.read_csv(CSV_FILE_NAME, sep=',', header=0)

In [15]:
import os, tempfile
tmp = tempfile.NamedTemporaryFile(delete=False)

try:
    print(tmp.name)
    with open(tmp.name, 'w') as outfile:
        outfile.write(csv_str)
    
    df = pd.read_csv(tmp.name, sep=',', header=0)
    
finally:
    #os.unlink(tmp.name) # remove the tmp file after use. 
    tmp.close()

/local/p25/fxz547/tmp/tmpap7y6j49


In [16]:
print (df.head())

  net   sta        date  clock_correction
0  7D  DE43  2012-11-27          1.039849
1  7D  DE43  2012-11-28          0.940850
2  7D  DE43  2012-11-29          0.841852
3  7D  DE43  2012-11-30          0.742853
4  7D  DE43  2012-12-01          0.643855


In [17]:
print (df.tail())

    net   sta        date  clock_correction
240  7D  DE43  2013-10-06          0.905443
241  7D  DE43  2013-10-07          0.903838
242  7D  DE43  2013-10-08          0.902233
243  7D  DE43  2013-10-09          0.900628
244  7D  DE43  2013-10-10          0.899023


In [20]:
df.describe()


clock_correction
count        245.000000
mean          -2.000434
std            3.793879
min          -11.302931
25%           -4.446864
50%           -0.063476
75%            0.963221
max            1.397653

In [23]:
df.dtypes

net                  object
sta                  object
date                 object
clock_correction    float64
dtype: object